# Pytorch DataSet and DataLoaders

## Building Up Stance

In [1]:
from sklearn.datasets import make_classification
import torch

In [2]:
# Creating a synthetic classification dataset using sklearn
X, y = make_classification(
    n_samples=10,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_classes=2,
    random_state=42
)

In [3]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

In [4]:
from torch.utils.data import Dataset, DataLoader

In [8]:
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    super().__init__()
    self.features = features
    self.labels = labels

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [9]:
dataset = CustomDataset(X, y)

In [13]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [16]:
for batch_features, batch_labels in dataloader:
  print(batch_features)
  print(batch_labels)
  print("-"*50)

tensor([[-0.5872, -1.9717],
        [-1.1402, -0.8388]])
tensor([0, 0])
--------------------------------------------------
tensor([[-0.7206, -0.9606],
        [-2.8954,  1.9769]])
tensor([0, 0])
--------------------------------------------------
tensor([[ 1.7273, -1.1858],
        [ 1.8997,  0.8344]])
tensor([1, 1])
--------------------------------------------------
tensor([[-1.9629, -0.9923],
        [-0.9382, -0.5430]])
tensor([0, 1])
--------------------------------------------------
tensor([[ 1.7774,  1.5116],
        [ 1.0683, -0.9701]])
tensor([1, 1])
--------------------------------------------------


## Improving Our Previous Code

In [17]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [18]:
# Making the dataframe using the csv file of Breast Cancer
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
# Presenting the Header of Dataframe
df.head()
# Header gives us good insights of the complete data means how many
# columns are there, which column is unnecessary or don't needed and
# which column needs some handling before use and so
# Like in this case we don't need 'id' and 'unnamed' columns so we will
# dropout them in near future

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [19]:
# Checking the Shape of Dataframe
df.shape

(569, 33)

In [20]:
df.drop(columns=['id','Unnamed: 32'], inplace=True)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [21]:
# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0], test_size=0.2)

In [22]:
# Scaling the data using StandarScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
# Printing the Scaled data and labels
print(X_train)
print(y_train)

[[ 1.71747586  0.08287821  1.61436354 ...  0.40755575  0.24984164
  -1.41287595]
 [ 0.03675669 -0.25908079 -0.03219822 ... -0.60004687 -0.41752174
  -0.4171269 ]
 [ 0.95118186  1.23498171  0.99504492 ...  0.85138072 -1.17567202
   0.51063928]
 ...
 [-0.75802406 -0.75021655 -0.78363813 ... -0.7472888  -0.07507991
   0.20385946]
 [-0.12561787 -0.74103644 -0.17175134 ... -0.68146476  0.67669936
  -0.40284479]
 [ 0.01681596  1.03531437  0.03592683 ...  0.92785056  2.90814589
   1.0590725 ]]
321    M
298    B
168    M
377    B
185    B
      ..
59     B
423    B
348    B
374    B
214    M
Name: diagnosis, Length: 455, dtype: object


In [24]:
# NOTE THE LABELS ARE OF 2 ALPHABET CLASSES (M AND B) SO FIRST
# ENCODING THEM USING LABELENCODER
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [25]:
# Now Printing the Coded Labels
print(y_train)

[1 0 1 0 0 0 0 1 1 1 0 1 1 1 1 1 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 1 1 0 1
 0 1 0 1 1 1 1 0 1 0 1 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0
 0 1 0 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1
 1 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 0 1 0 0
 0 1 0 1 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0 0 0 1 1 0
 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 0 0
 0 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 1 1 0 1 0 0 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 1
 0 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0
 0 1 0 1 1 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0
 1 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 1 1 0 1 0 0 0 1 1 0 0 0
 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 0 0
 0 1 1 1 1 0 0 0 0 0 1]


In [26]:
# NOW WE WANT TO MAKE OUR MODEL IN PYTORCH BUT WE HAVE EVERYTHING IN
# NUMPY ARRAYS SO CONVERTING THEM
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [27]:
# Now Printing the Shapes of the tensors
print(X_train_tensor.shape)
print(X_test_tensor.shape)
print(y_train_tensor.shape)
print(y_test_tensor.shape)

torch.Size([455, 30])
torch.Size([114, 30])
torch.Size([455])
torch.Size([114])


In [28]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, features, labels):
    super().__init__()
    self.features = features
    self.labels = labels

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [30]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [32]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [38]:
import torch.nn as nn

class MySimpleNN(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):
    out = self.linear(features)
    out = self.sigmoid(out)
    return out


In [39]:
# Initializing the Important Parameters for our model
# Learning Rate - Rate at which gradient descent or any other optimization technique
#                 should proceed to minimize Loss Function
lr = 0.1
# Epochs - No of Iterations for which model is run (usually for trained)
epochs = 25


In [40]:
# Model Creation
model = MySimpleNN(X_train_tensor.shape[1])

In [41]:
loss_function = nn.BCELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [46]:
for epoch in range(epochs):
  for batch_features, batch_labels in train_loader:
    y_pred = model(batch_features.float())
    loss = loss_function(y_pred, batch_labels.view(-1,1).float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(f"Epoch: {epoch + 1}, Loss: {loss.item()}")

Epoch: 1, Loss: 0.6995464563369751
Epoch: 2, Loss: 0.4174444377422333
Epoch: 3, Loss: 0.40389785170555115
Epoch: 4, Loss: 0.3498816192150116
Epoch: 5, Loss: 0.2072148472070694
Epoch: 6, Loss: 0.2938174605369568
Epoch: 7, Loss: 0.2677668631076813
Epoch: 8, Loss: 0.20401838421821594
Epoch: 9, Loss: 0.20093610882759094
Epoch: 10, Loss: 0.30565720796585083
Epoch: 11, Loss: 0.10418287664651871
Epoch: 12, Loss: 0.2367384284734726
Epoch: 13, Loss: 0.12454796582460403
Epoch: 14, Loss: 0.09467753022909164
Epoch: 15, Loss: 0.19377729296684265
Epoch: 16, Loss: 0.1392284333705902
Epoch: 17, Loss: 0.27943989634513855
Epoch: 18, Loss: 0.09690098464488983
Epoch: 19, Loss: 0.05673474073410034
Epoch: 20, Loss: 0.17149941623210907
Epoch: 21, Loss: 0.05097758024930954
Epoch: 22, Loss: 0.12936656177043915
Epoch: 23, Loss: 0.39796844124794006
Epoch: 24, Loss: 0.04572045058012009
Epoch: 25, Loss: 0.094406358897686


In [47]:
# NOW EVALUATING THE MODEL BY TESTING IT WITH OUR TESTING DATA
model.eval()
accuracy_list = []
with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    y_pred = model(batch_features.float())
    y_pred = (y_pred > 0.8).float()
    # Accuracy is obtained manually
    accuracy = (y_pred == batch_labels).float().mean().item()
    accuracy_list.append(accuracy)

overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f"Overall Accuracy: {overall_accuracy}")

Overall Accuracy: 0.5505280643701553
